<img align="left" src="https://github.com/Db2-DTE-POC/CPDDVHOL4/blob/main/media/Assets&ArchHeader.jpeg?raw=true">

# Using the Db2 Console RESTful Service Class

### Db2 Console Class for Cloud Pak for Data
This Jupyter Notebook uses a reusable Python class library that encapsulates come best practices of how to use the Open APIs that are available for Db2 running in Cloud Pak for Data. Everything in the Db2 Console is available through an open RESTful Services API. The full set of APIs are documented as part of the Db2 Data Management Console user interface. 

### Where to find this sample online
You can find a copy of this notebook at https://github.com/Db2-DTE-POC/CPDDVHOL4

Let's get started by loading the db2console.ipynb class library notebook, which is also available on GIT. The commands below copy the reusable library from GIT onto the local Cloud Pak for Data filesystem and runs the python file to create the Db2 Console API Class. To check out the reusable code on GIT click the follwing link: https://github.com/Db2-DTE-POC/CPDDVHOL4/blob/main/Db2ConsoleAPIClassforCPD.ipynb

In [ ]:
!wget -O Db2ConsoleAPIClassforCPD.ipynb https://raw.githubusercontent.com/Db2-DTE-POC/CPDDVHOL4/main/Db2ConsoleAPIClassforCPD.ipynb
%run Db2ConsoleAPIClassforCPD.ipynb
print('Db2ConsoleAPIClassforCPD.ipynb loaded')

### Db2 Data Management Console Connection
The first step is to create an instance of the Db2Console class (a Python object). The next cell creates an object called **databaseAPI**. The rest of this lab calls functions that are part of that object.

The **databaseAPI** object creation requires the URL of the Cloud Pak for Data Console as well as the name of the Data Management Console instance running on Cloud Pak for Data. To authenticate a connection to the console API you also need a valid Cloud Pak for Data userid and password as well as a default database instance to connect to.

In [ ]:
# Set the service URL to connect from inside the ICPD Cluster
Console  = 'https://cpd-cpd-instance.apps.demo.ibmdte.net:31192'

# Connect to the Db2 Data Management Console service
user     = 'admin'
password = 'CP4DDataFabric'

# Set up the required connection
databaseAPI = Db2Console(Console, 'dmc-1635311028943779')
api = '/v1'
databaseAPI.authenticate(api, user, password, 'db2wh-1635951043918331')
database = Console

If the connection was successfully established, the new object contains a reusable token that is used to reconnect to the console API service for each function call. You don't need to ever use the token in your code, but if you want to see what a secure token looks like run the next cell.

In [ ]:
databaseAPI.getBearerToken()

### Confirm the connection
To confirm that your connection is working you can list the Console connection profiles. Each profile represents a connection to one of the available Db2 Warehouse, Db2 OLTP or Data Virtualization databases in Cloud Pak for Data.

In [ ]:
databaseAPI.getConnectionProfiles()

### Catalog Functions
Now that you are connected to a specific database, in this example 'ONTIME', you can call functions that let you access catalog information. You can get a list of the schemas in the Ontime database. The cell below retrieves all the rows in a dataframe and displays the first 5.

In [ ]:
databaseAPI.getSchemas().head(5)

#### Search and Count Tables and Views
You can also use capabilities that are built into the console. For example you can find out how many tables include the text "AIRCRAFT" or seach all the views (both user and catalog views) that include the text "TABLES". By default the functions below only search user tables. Adding "true" to the function call also searches the system tables. Try running the cell below. Then try changing "true" to "false" and see the different result.

In [ ]:
display(databaseAPI.getSearchTableList("AIRCRAFT"))
display(databaseAPI.getSearchViewList("TABLE", "true"))

#### Tables in a Schema
The next function all returns the first five tables contained in the "ONTIME" schema

In [ ]:
databaseAPI.getTablesInSchema("ONTIME").head(5)

#### Fuzy object search
The next function returns a list of either the tables or views that match search text. You can specify the number of rows in the result set (in this example 5) and specify whether you want to search user object or user and system objects (in this example true means searching both). 

In [ ]:
display(databaseAPI.searchObjects('view',"TABLE",5,'true'))
display(databaseAPI.searchObjects('table',"AIRLINE",5,'true'))

### Running Scripts and Workloads
The Db2Console class can also call the SQL Editor service to run Db2 scripts. This isn't limited to single SQL statments. Scripts that include multiple statements are also supported. 

To make it easy to run the same script against different databases, the fucntion call requires the connection profile name, the userid and password and the sql script text. The next cell runs three SQL statements. The database connection and authentication is included in the call. 

It returns a JSON string that include details on each statement, its runtime, column types, the limit of returned rows, the full row count in the result set, and the actual results to the row limit. 

In [ ]:
sql = \
'''
SELECT TABSCHEMA, TABNAME, STATUS FROM SYSCAT.TABLES; 
SELECT VIEWSCHEMA, VIEWNAME, VALID FROM SYSCAT.VIEWS;
SELECT TABSCHEMA, TABNAME, COLNAME, TYPENAME, LENGTH FROM SYSCAT.COLUMNS
'''
user = 'admin'
password = 'CP4DDataFabric'
profile = 'dv-1635944153872816'

display(databaseAPI.runScript(profile, user, password, sql))

To make it easier to see the results, the displayResults function parses the JSON into a readable format.

#### Formatting Results

In [ ]:
databaseAPI.displayResults(databaseAPI.runScript(profile, user, password, sql))

The number of cells returned is limited to 10 by default. You can add an additional parameter to the runScript command to return a much larger result set, which you can then manipulate in Python. The returnRows function converts the JSON result into a dataframe. It requires the json that is returned by runScipt and the index of the SQL result you want to work with.

#### Returning Results as Dataframes

In [ ]:
json = databaseAPI.runScript(profile, user, password, sql, 10000)
df = databaseAPI.returnRows(json,0)
display(df.head(5))
display(df.tail(5))

In [ ]:
df = databaseAPI.returnRows(json,1)
display(df.head(5))
display(df.tail(5))

In [ ]:
df = databaseAPI.returnRows(json,2)
display(df.head(5))
display(df.tail(5))

#### Running Workloads and Measuring Results

One of the most powerful functions built into the Db2Console class is **runWorkload**. It lets you run multiple scripts against multiple databases in a loop. This is particularly useful for demonstrating Db2 monitoring or for measuring the performance of SQL against different databases.

In the next example, two scripts are run repeatedly against all the databases currently cataloged by the Db2 Console.

The runtimes are collected along the way and returned in a dataframe.

In [ ]:
profileList = ['db2oltp-1636379315142134','db2oltp-1635953643145137','db2wh-1635951043918331']
sql1 = \
'''
SELECT TABSCHEMA, TABNAME, STATUS FROM SYSCAT.TABLES; 
SELECT VIEWSCHEMA, VIEWNAME, VALID FROM SYSCAT.VIEWS;
'''
sql2 = \
'''
SELECT TABSCHEMA, TABNAME, COLNAME, TYPENAME, LENGTH FROM SYSCAT.COLUMNS
'''
user = 'admin'
password = 'CP4DDataFabric'
scriptList = [sql1, sql2]
profileReps = 2
scriptReps = 2
pause = 0.25

df = databaseAPI.runWorkload(profileList, scriptList, user, password, profileReps, scriptReps,pause)

display(df)

Now we can use the results in the dataframe to look at the results statistically. First we can see the average runtime for each statement across the databases.

In [ ]:
print('Mean runtime in ms')
pd.set_option('display.max_colwidth', 100)
stmtMean = df.groupby(['statement']).mean()
print(stmtMean)

We can also display the total runtime for each statement across databases.

In [ ]:
print('Total runtime in ms')
pd.set_option('display.max_colwidth', 100)
stmtSum = df.groupby(['statement']).sum()
print(stmtSum)

We can even graph the total run time for all the statements can compare database performance. Since there are more rows in the Db2 Warehouse database tables the statements may take longer to return a result. 

In [ ]:
print('Mean runtime in ms')
pd.set_option('display.max_colwidth', 100)
profileSum = df.groupby(['profile']).sum()
profileSum.plot(kind='bar')
plt.show()

Of course you can also analyze the slowest database by using some simple dataframe functions. The slowest database has the maximum total elapsed runtime. 

In [ ]:
print('Slowest Database')
slowestProfile = profileSum['runtime_ms'].idxmax()
print(slowestProfile)

#### Additional Options
Just like using the SQL Editor you can also specify whether to stop the script if it encounters and error or to continue. You can also specify the seperator character between individual statements. Here is the full function call with defaults and options:

    runScript(profile, user, password, sqlText, limit=10, separator=';', stopOnError=False):

#### ADMIN_CMD Commands and Calling Stored Procedures
The SQL Editor can also be used to execute stored procedure calls. In this example, the procedure call updates statistics on the STOCKS.CUSTOMER table in the STOCKs OLTP database

In [ ]:
sql = \
'''
CALL SYSPROC.ADMIN_CMD ('RUNSTATS ON TABLE STOCKS.CUSTOMER ON KEY COLUMNS and INDEXES ALL'); 
'''
user = 'admin'
password = 'CP4DDataFabric'
profile = 'db2oltp-1635953643145137'

display(databaseAPI.runScript(profile, user, password, sql))

### Current Metrics Functions
Now that you can run a workload to exercise Db2, you can measure what is going on. The following function calls let you see what applications are connected to the "Ontime" database, see any statements that are currently in-flight and see the frequently used statements stored in the Db2 package cache.

The includeSystem parameter defines whether applications or statements generated by Db2 itself or the Db2 Console are included in the results. Let's include all statements. 

In [ ]:
includeSystem = "true"

The next cells likes the applications that are currently connected to the database

In [ ]:
databaseAPI.getCurrentApplicationsConnections(includeSystem)

This next cell lists any statements currently running

In [ ]:
databaseAPI.getInflightCurrentList(includeSystem)

Finally this example returns the list of every statement currently in the Db2 Package Cache. This gives a good representation of statements that are frequently used. 

In [ ]:
databaseAPI.getCurrentPackageCacheList(includeSystem).head(5)

### Timeseries Monitoring Functions
One of the key capabilities of the Db2 Console is that is collects historical monitoring information as timeseries data. Each of the examples below has a parallel page in the Db2 Console. 

The next set of functions returns monitoring data based on a start and endtime. 

The console and Db2 use EPOCH time, which is the number of milli-seconds since January 1st 1970. The cell below sets startTime and endTime. endTime is the current and end time. startTime is set to 12 hours earlier. 

In [ ]:
import time
from datetime import date
oneHour = 3600000
endTime = int(time.time())*1000
startTime = endTime-(oneHour*12)

#### Time Based Metrics - Summary Functions
The following functions return a total summary of the number of user statements that have run over the last 12 hours as well as the average response time in ms over that same period. 

In [ ]:
databaseAPI.getStatementsCount(startTime, endTime)

In [ ]:
databaseAPI.getAverageResponseTime(startTime, endTime)

#### Time Based Metrics - Interval Measurement Functions
The following functions return a measurement for each monitoring interval over the last 12 hours. The examples below return average response time and total rows read during each monitoring interval. The last 5 intervals are displayed

In [ ]:
databaseAPI.getResponseTime(startTime, endTime).tail(5)

In [ ]:
databaseAPI.getRowsRead(startTime, endTime).tail(5)

#### Time Based Metrics - Object Functions
The following functions return monitoring data over the last 12 hours with a summary row for each object. The latest 5 entries are displayed. This first call returns metrics for tables used in the last 12 hours.

In [ ]:
databaseAPI.getTablesMetrics(startTime, endTime, includeSystem).tail(5)

The next statement returns details of individual statements that ran over the last twelve hours. 

In [ ]:
databaseAPI.getIndividualStatementExecution(startTime, endTime).tail(5)

Finally this statement returns a history of the statements that were found in the package cache over the last 12 hours. 

In [ ]:
databaseAPI.getPackageCacheStatement(startTime, endTime, includeSystem).tail(5)

## What's Next
Try creating your own workload in a seperate notebook that takes longer to run. Check out the ONTIME project for heavier workloads. You can even use the workload example above and increase the number of repetitions. In a seperate notebook, try running the monitoring functions or use the Db2 Console to see how Db2 responds under load.

You can also explore the https://github.com/Db2-DTE-POC/CPDDVHOL4/blob/main/Db2ConsoleAPIClassforCPD.ipynb library. There are additional function calls you can explore and experiment with.

#### Credits: IBM 2019-2021, Peter Kohlmann [kohlmann@ca.ibm.com]